In [1]:
import os,sys,inspect
import numpy as np
import torch
import torch.nn as nn
import cv2
import torch.nn.functional as F

#os.environ["CUDA_VISIBLE_DEVICES"]="1"

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm

from lib.hybridnet.modules.v2vnet.repro_layer import ReprojectionLayer
from lib.dataset.dataset3D import Dataset3D
from lib.hybridnet.modules.efficienttrack.model import EfficientTrackBackbone
import lib.hybridnet.modules.efficienttrack.darkpose as darkpose
from lib.hybridnet.utils import ReprojectionTool
from lib.hybridnet.modules.v2vnet.model import V2VNet
from lib.config.project_manager import ProjectManager
from lib.hybridnet.model import HybridNetBackbone

project = ProjectManager()
project.load('12Cam_wo_Ben')
cfg = project.get_cfg()


384
Successfully loaded project 12Cam_wo_Ben!


In [2]:
test_set = Dataset3D(cfg = cfg, set='val')
hybridNet = HybridNetBackbone(cfg, test_set.coco.dataset['calibration']['intrinsics'], 
                        test_set.coco.dataset['calibration']['extrinsics']).cuda()

weights = '/home/trackingsetup/Documents/Vortex/projects/12Cam_wo_Ben/models/vortex/Run_20211012-152447/Vortex-d_55.pth'
hybridNet.load_state_dict(torch.load(weights), strict = True)

reproTool = ReprojectionTool('Camera_T', test_set.root_dir, test_set.coco.dataset['calibration']['intrinsics'], test_set.coco.dataset['calibration']['extrinsics'])

hybridNet.requires_grad_(False)
hybridNet.eval()
hybridNet = hybridNet.cuda()




line_idxs = [[0,1], [1,2], [2,3], [4,5], [5,6], [6,7], [8,9], [9,10], [10,11], [12,13], [13,14], [14,15], [16,17], [17,18], [18,19], [3,7], [7,11], [11,15], [3,21], [7,21],[11,22], [15,22],[21,22], [18,15], [19,22]]

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [3]:
import time

pointsNet = []
points2DNet = []
pointsGT = []
tot_errors_hm3d = []
tot_errors_class = []
joint_lengths_hm3d = [[] for i in range(15)]
joint_lengths_class = [[] for i in range(15)]

print (len(test_set.image_ids))
for item in range(len(test_set.image_ids)):
    image_info = test_set.coco.loadImgs(test_set.image_ids[item])[0]
    sample = test_set.__getitem__(item)
    keypoints3D = sample[1]
    imgs = torch.from_numpy(np.array(sample[0])).cuda()
    imgs_p = torch.unsqueeze(imgs.permute(0,3,1,2).float(),0)
    centerHM = torch.cuda.IntTensor(sample[2])
    center3D = torch.cuda.FloatTensor(sample[3])

    start_time = time.time()
    heatmap3D, heatmaps_padded, points3D_net = hybridNet(imgs_p,torch.unsqueeze(centerHM,0),  torch.unsqueeze(center3D,0))

    print ((time.time()-start_time)*1000)

    preds, maxvals = darkpose.get_final_preds(heatmaps_padded[0].clamp(0,255).cpu().numpy(), None)
    preds *= 2

    points3D_rec = []
    for i in range(len(preds[0])):
        camsToUse = []
        for cam in range(12):
            if maxvals[cam][i] > 60:
                camsToUse.append(cam)
        point = reproTool.reconstructPoint(np.transpose(preds[:,i,:]), camsToUse)
        points3D_rec.append(point)


    heatmap3D = heatmap3D.cpu().numpy()[0]
    center3D = center3D.cpu().numpy()
    points3D_net = points3D_net[0].cpu().detach().numpy()

    for i,idx in enumerate(line_idxs[0:15]):
        joint_lengths_hm3d[i].append(np.linalg.norm(np.array(points3D_net[idx[0]]-points3D_net[idx[1]])))
        joint_lengths_class[i].append(np.linalg.norm(np.array(keypoints3D[idx[0]]-keypoints3D[idx[1]])))

    pointsNet.append(points3D_net)
    points2DNet.append(points3D_rec)
    pointsGT.append(keypoints3D)

    errors_3dHM = []
    errors_Std = []
    for i in range(23):
        if i != 20:
            errors_3dHM.append(np.linalg.norm(np.array(points3D_net[i])-keypoints3D[i]))
            errors_Std.append(np.linalg.norm(np.array(points3D_rec[i])-keypoints3D[i]))
    print ('3DHM:', np.mean(np.array(errors_3dHM)))
    print ('Std:', np.mean(np.array(errors_Std)))

    tot_errors_hm3d.append(np.mean(np.array(errors_3dHM)))
    tot_errors_class.append(np.mean(np.array(errors_Std)))

92
738.9011383056641
3DHM: 2.243641374856648
Std: 8.022118140670473
145.4777717590332
3DHM: 2.3169526995814076
Std: 4.824802042822451
179.80432510375977
3DHM: 2.7104003467076114
Std: 6.520627227549733
172.79911041259766
3DHM: 3.090823086574252
Std: 6.794494473133256
149.84869956970215
3DHM: 3.108686228089003
Std: 7.194149467651791
106.4450740814209
3DHM: 2.0939265091490706
Std: 4.774379217935229
157.08470344543457
3DHM: 3.9421019230598877
Std: 12.739587961468258
164.05677795410156
3DHM: 4.289520849357052
Std: 12.991616405738277
167.50168800354004
3DHM: 3.6714018725894744
Std: 8.232083269821047
169.1734790802002
3DHM: 2.7589121890531314
Std: 6.1428356233693755
155.0009250640869
3DHM: 4.262981713774393
Std: 10.584334861776012
147.48334884643555
3DHM: 8.180141701118073
Std: 13.863124891222652
168.77079010009766
3DHM: 3.4254894808535714
Std: 6.654190774724716
118.89314651489258
3DHM: 2.9642535938317023
Std: 4.406972519577678
138.32879066467285
3DHM: 4.198755769745995
Std: 10.10055308271076

In [4]:
print ("Var HM3D:", np.mean(np.var(np.array(joint_lengths_hm3d), axis = 1)))
print ("Var Class:", np.mean(np.var(np.array(joint_lengths_class), axis = 1)))
print ("Total 3DHM:", np.mean(tot_errors_hm3d))
print ("Total Classic:", np.mean(tot_errors_class))

from numpy import savetxt
#savetxt('csvs/points3DNet.csv', np.array(pointsNet).reshape((-1, 23*3)), delimiter=',')
#savetxt('csvs/points2DNet.csv', np.array(points2DNet).reshape((-1, 23*3)), delimiter=',')
#savetxt('csvs/pointsGT.csv', np.array(pointsGT).reshape((-1, 23*3)), delimiter=',')

Var HM3D: 5.060286
Var Class: 6.4106291615093705
Total 3DHM: 2.8941206151889576
Total Classic: 6.3061051589091655
